In [4]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from numpy import linspace
import argparse
import cv2 as cv
import glob
from PIL import Image

Hs = np.load("run/numpy/Hs.npy")

def getK(Hs):
    V = np.zeros((2*len(Hs),6))
    for idx, H in enumerate(Hs):
        h1, h2 = H[:,0], H[:,1]

        # h1^T*B*h2 = 0
        v1 = [h1[0]*h2[0],                  # h10h20
            h1[1]*h2[0]+h1[0]*h2[1],        # h11h20+h10h21
            h1[2]*h2[0]+h1[0]*h2[2],        # h12h20+h10h22
            h1[1]*h2[1],                    # h11h21
            h1[2]*h2[1]+h1[1]*h2[2],        # h12h21+h11h22
            h1[2]*h2[2]                     # h12h22
        ]
        # h1^T*B*h1 = h2^T*B*h2
        v2 = [h1[0]**2-h2[0]**2,            # h10^2 - h20^2
            2*(h1[0]*h1[1]-h2[0]*h2[1]),    # 2(h10h11-h20h21)
            2*(h1[0]*h1[2]-h2[0]*h2[2]),    # 2(h10h12-h20h22)
            h1[1]**2-h2[1]**2,              # h11^2-h21^2
            2*(h1[1]*h1[2]-h2[1]*h2[2]),    # 2(h11h12-h21h22)
            h1[2]**2-h2[2]**2]              # h12^2-h22^2

        # put v1 and v2 into V
        V[2*idx] = np.array(v1)
        V[2*idx+1] = np.array(v2)

    # solve b = argmin(Vb)
    U, D, VT = np.linalg.svd(V)
    b11, b12, b13, b22, b23, b33 = VT[-1]

    # reconstruct B from b
    B = np.array([[b11,b12,b13],[b12,b22,b23],[b13,b23,b33]])

    # make it positive definite (if it isn't yet)
    if(B[0][0] < 0):
        B = -B

    # B = L*L^T
    L = np.linalg.cholesky(B)

    # L = K^-1^T
    K = np.linalg.inv(np.transpose(L))

    # normalize
    K /= K[2][2]

    K[0][1] = 0

    print(K)
    return K
K=getK(Hs)
np.save('drone_calib_mtx.npy',K)

[[1.18162341e+03 0.00000000e+00 4.79134393e+02]
 [0.00000000e+00 6.86457797e+02 8.79928518e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [12]:
import matplotlib
#matplotlib.use('Agg')
import cv2
import glob
import sys
import datetime
import numpy as np
import pickle
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from scipy.spatial.transform import Rotation as R
from scipy import interpolate
import mpl_toolkits.mplot3d.art3d as art3d
import time
print(cv2.__version__)
def vid2frame(file_name,outputFolder):#抓video中所有的frame
    vid_L = cv2.VideoWriter('track_restlt.MP4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (1280, 720))
    coord=np.load("track.npy")
    count = 0

    print("\nProcessing {} ...".format(file_name))
    #vidcap = cv2.VideoCapture("run/videos/{}/{}".format(date, file_name))
    vidcap = cv2.VideoCapture("{}".format(file_name))

    fps = vidcap.get(cv2.CAP_PROP_FPS)
    length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    while vidcap.isOpened():
        count += 1
        
        # ret will return true if a frame is read successfully
        ret, frame = vidcap.read()
        
        if ret:
            sys.stdout.write("\rExtracting frame {:04d} ...".format(count))
            sys.stdout.flush()
            if count>200:
                for c in coord[200:count]:
                    cv2.circle(frame, [c[0]+c[2]//2,c[1]+c[3]//2], 1,(0, 0, 255), 1)
            #cv2.imwrite("run/frames/{}_frame{:04d}.png".format(file_name, count), frame)
            #cv2.imwrite(outputFolder+"/{}_frame{:04d}.png".format(file_name, count), frame)
            vid_L.write(frame)
        else:
            vid_L.release()

            print("\nExtraction finished!")
            break

    vidcap.release()
    return fps, length
outputFolder="result/frame"
file_name="result.mp4"
fps, length = vid2frame(file_name,outputFolder)
print(fps)

4.6.0

Processing result.mp4 ...
Extracting frame 1501 ...
Extraction finished!
30.0


In [7]:
coord=np.load("track.npy")
print(len(coord))

1501


In [6]:
shape = (720, 1280, 3) # y, x, RGB
origin_img = np.zeros(shape, np.uint8)
for c in coord:
    cv2.circle(origin_img, [c[0],c[1]], 1,(0, 0, 255), 1)
cv2.imshow("test",origin_img)
cv2.waitKey(0)
vid_L = cv2.VideoWriter('run/output_videos/drone_{}L.MP4'.format(time.strftime("%m_%d_%H_%M_%S",time.localtime())), cv2.VideoWriter_fourcc(*'mp4v'), 30, (960, 720))


-1

In [10]:
for c in coord:
    print([c[0]+c[2]/2,c[1]+c[3]/2])

[1057.5, 511.0]
[1057.5, 511.0]
[1057.5, 511.0]
[1057.5, 511.0]
[1057.5, 511.0]
[1057.5, 511.0]
[1056.5, 511.0]
[1057.5, 511.0]
[1057.5, 511.0]
[1056.5, 511.0]
[1057.5, 511.0]
[1056.5, 511.0]
[1056.5, 511.0]
[1056.5, 511.0]
[1056.5, 511.0]
[1056.5, 511.0]
[1056.5, 511.0]
[1056.5, 511.0]
[1056.5, 511.0]
[1056.5, 511.0]
[1056.5, 511.0]
[1055.5, 511.0]
[1056.5, 511.0]
[1055.5, 511.0]
[1055.5, 511.0]
[1055.5, 511.0]
[1055.5, 512.0]
[1055.5, 512.0]
[1055.5, 512.0]
[1055.5, 512.0]
[1055.5, 512.0]
[1054.5, 512.0]
[1055.5, 512.0]
[1054.5, 512.0]
[1055.5, 512.0]
[1054.5, 512.0]
[1054.5, 512.0]
[1054.5, 512.0]
[1054.5, 512.0]
[1054.5, 512.0]
[1054.5, 512.0]
[1053.5, 512.0]
[1054.5, 512.0]
[1054.5, 512.0]
[1053.5, 512.0]
[1054.5, 512.0]
[1054.5, 511.0]
[1054.5, 511.0]
[1055.5, 511.0]
[1055.5, 511.0]
[1057.5, 509.0]
[1058.5, 509.0]
[1058.5, 509.0]
[1059.5, 508.0]
[1060.5, 508.0]
[1063.5, 507.0]
[1064.5, 506.0]
[1064.5, 507.0]
[1065.5, 507.0]
[1066.5, 505.0]
[1069.5, 504.0]
[1070.5, 503.0]
[1071.5,